<a href="https://colab.research.google.com/github/TanyaGupta37/music-emotion-recognition-using-emotify-/blob/main/Emotify_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Check what's in your Drive
import os

print("Checking your Google Drive structure...")
print("\n1. Root of Drive:")
!ls /content/drive/MyDrive/ | head -20

print("\n2. Looking for emotify_colab folder:")
!ls /content/drive/MyDrive/ | grep emotify

print("\n3. If emotify_colab exists, checking inside:")
if os.path.exists('/content/drive/MyDrive/emotify_colab'):
    !ls -R /content/drive/MyDrive/emotify_colab/
else:
    print("⚠️ emotify_colab folder NOT FOUND in Drive!")

Checking your Google Drive structure...

1. Root of Drive:
Colab Notebooks
emotify_colab

2. Looking for emotify_colab folder:
emotify_colab

3. If emotify_colab exists, checking inside:
/content/drive/MyDrive/emotify_colab/:
code  data  results

/content/drive/MyDrive/emotify_colab/code:
dl_models.py

/content/drive/MyDrive/emotify_colab/data:
audio		      emotify_labels_5class.csv
emotify_features.csv  emotify_labels_matched.csv

/content/drive/MyDrive/emotify_colab/data/audio:
emotify

/content/drive/MyDrive/emotify_colab/data/audio/emotify:
classical  electronic  pop  rock

/content/drive/MyDrive/emotify_colab/data/audio/emotify/classical:
100.mp3  19.mp3  28.mp3  37.mp3  46.mp3  55.mp3  64.mp3  73.mp3  82.mp3  91.mp3
10.mp3	 1.mp3	 29.mp3  38.mp3  47.mp3  56.mp3  65.mp3  74.mp3  83.mp3  92.mp3
11.mp3	 20.mp3  2.mp3	 39.mp3  48.mp3  57.mp3  66.mp3  75.mp3  84.mp3  93.mp3
12.mp3	 21.mp3  30.mp3  3.mp3	 49.mp3  58.mp3  67.mp3  76.mp3  85.mp3  94.mp3
13.mp3	 22.mp3  31.mp3  40.mp3  4.

In [3]:
# Cell 2: Copy files from Drive

import os

# Create directories first
os.makedirs('data', exist_ok=True)
os.makedirs('code', exist_ok=True)

!cp /content/drive/MyDrive/emotify_colab/data/emotify_features.csv data/
!cp /content/drive/MyDrive/emotify_colab/data/emotify_labels_matched.csv data/
!cp /content/drive/MyDrive/emotify_colab/code/dl_models.py code/

print("✓ Files copied from Drive!")


✓ Files copied from Drive!


In [4]:
# Cell 3: Install packages
!pip install -q scikit-learn pandas numpy tensorflow
print("✓ Packages installed!")


✓ Packages installed!


In [5]:
# Cell 4: Load and verify data
import pandas as pd
import numpy as np

features_df = pd.read_csv('data/emotify_features.csv')
labels_df = pd.read_csv('data/emotify_labels_matched.csv')

print("="*60)
print("DATA LOADED")
print("="*60)
print(f"Features shape: {features_df.shape}")
print(f"Labels shape: {labels_df.shape}")
print(f"\nEmotion distribution:")
print(features_df['emotion'].value_counts())

# Prepare data
X = features_df.drop('emotion', axis=1).values
y = features_df['emotion'].values

print(f"\nX shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"Unique classes: {len(np.unique(y))}")

DATA LOADED
Features shape: (399, 150)
Labels shape: (399, 2)

Emotion distribution:
emotion
relaxing      69
anxious       54
amusing       49
happy         41
energizing    38
annoying      36
dreamy        35
sad           30
neutral       27
joyful        20
Name: count, dtype: int64

X shape: (399, 149)
y shape: (399,)
Unique classes: 10


In [11]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, label_binarize
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

# ------------------------------------------------------------
# Encode labels if needed
# ------------------------------------------------------------
if y.dtype == object:
    le = LabelEncoder()
    y_enc = le.fit_transform(y)
else:
    y_enc = y

num_classes = len(np.unique(y_enc))

# ------------------------------------------------------------
# Models (simple, like earlier pipeline)
# ------------------------------------------------------------
knn_model = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=11, weights='distance'))
])

svm_model = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(
        kernel='rbf',
        C=10,
        gamma='scale',
        probability=True,
        class_weight='balanced'
    ))
])

models = {
    "KNN": knn_model,
    "SVM": svm_model
}

# ------------------------------------------------------------
# Evaluation (same style as earlier models)
# ------------------------------------------------------------
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

results = {}

for name, model in models.items():
    print("\n" + "="*60)
    print(f"TRAINING {name}")
    print("="*60)

    accs, aucs, f1s = [], [], []

    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y_enc), 1):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y_enc[train_idx], y_enc[val_idx]

        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        y_proba = model.predict_proba(X_val)

        acc = accuracy_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred, average='macro')

        y_val_bin = label_binarize(y_val, classes=range(num_classes))
        auc = roc_auc_score(
            y_val_bin,
            y_proba,
            average='macro',
            multi_class='ovr'
        )

        accs.append(acc)
        f1s.append(f1)
        aucs.append(auc)

        print(
            f"Fold {fold:2d} | "
            f"Acc: {acc:.4f} | "
            f"AUC: {auc:.4f} | "
            f"F1: {f1:.4f}"
        )

    print("-"*60)
    print(f"{name} FINAL RESULTS")
    print("-"*60)
    print(f"Accuracy:  {np.mean(accs):.4f} ± {np.std(accs):.4f}")
    print(f"ROC-AUC:   {np.mean(aucs):.4f} ± {np.std(aucs):.4f}")
    print(f"F1-Score:  {np.mean(f1s):.4f} ± {np.std(f1s):.4f}")

    results[name] = {
        'accuracy_mean': np.mean(accs),
        'accuracy_std': np.std(accs),
        'auc_mean': np.mean(aucs),
        'auc_std': np.std(aucs),
        'f1_mean': np.mean(f1s),
        'f1_std': np.std(f1s),
    }

# ------------------------------------------------------------
# Summary (matches earlier benchmark style)
# ------------------------------------------------------------
print("\n" + "="*60)
print("BASELINE RESULTS (EARLIER PIPELINE)")
print("="*60)
for model, res in results.items():
    print(
        f"{model:<4} | "
        f"AUC: {res['auc_mean']:.4f} ± {res['auc_std']:.4f} | "
        f"F1: {res['f1_mean']:.4f} ± {res['f1_std']:.4f}"
    )
print("="*60)



TRAINING KNN
Fold  1 | Acc: 0.1250 | AUC: 0.4981 | F1: 0.1111
Fold  2 | Acc: 0.2000 | AUC: 0.5613 | F1: 0.1298
Fold  3 | Acc: 0.2500 | AUC: 0.5473 | F1: 0.1578
Fold  4 | Acc: 0.1250 | AUC: 0.5086 | F1: 0.0711
Fold  5 | Acc: 0.1250 | AUC: 0.5109 | F1: 0.0994
Fold  6 | Acc: 0.2000 | AUC: 0.5884 | F1: 0.1305
Fold  7 | Acc: 0.0250 | AUC: 0.5040 | F1: 0.0125
Fold  8 | Acc: 0.2000 | AUC: 0.5600 | F1: 0.1640
Fold  9 | Acc: 0.1500 | AUC: 0.5740 | F1: 0.1405
Fold 10 | Acc: 0.1282 | AUC: 0.5409 | F1: 0.1051
------------------------------------------------------------
KNN FINAL RESULTS
------------------------------------------------------------
Accuracy:  0.1528 ± 0.0595
ROC-AUC:   0.5394 ± 0.0305
F1-Score:  0.1122 ± 0.0425

TRAINING SVM
Fold  1 | Acc: 0.0750 | AUC: 0.6186 | F1: 0.0448
Fold  2 | Acc: 0.1750 | AUC: 0.5520 | F1: 0.1392
Fold  3 | Acc: 0.0500 | AUC: 0.5673 | F1: 0.0251
Fold  4 | Acc: 0.1750 | AUC: 0.5638 | F1: 0.0920
Fold  5 | Acc: 0.1500 | AUC: 0.5765 | F1: 0.1150
Fold  6 | Acc: 0

In [12]:
# Cell 5: Import models and train
import sys
sys.path.append('code')
from dl_models import build_mlp_model, build_deeper_mlp, evaluate_model

print("="*60)
print("STARTING TRAINING")
print("="*60)

# Train MLP (3-layer)
print("\n🚀 Training Model 1/2...")
results_mlp = evaluate_model(
    build_mlp_model,
    X, y,
    "MLP (3-layer)",
    n_splits=10
)

# Train Deeper MLP (5-layer with BatchNorm)
print("\n🚀 Training Model 2/2...")
results_deep_mlp = evaluate_model(
    build_deeper_mlp,
    X, y,
    "Deep MLP (5-layer + BatchNorm)",
    n_splits=10
)

# Summary
import pandas as pd
results_df = pd.DataFrame([results_mlp, results_deep_mlp])

print("\n" + "="*60)
print("COMPARISON WITH BASELINE")
print("="*60)

baseline_knn = 0.5394
baseline_svm = 0.5716
baseline_best = max(baseline_knn, baseline_svm)

print(f"Baseline (KNN):  {baseline_knn:.4f} ROC-AUC")
print(f"Baseline (SVM):  {baseline_svm:.4f} ROC-AUC")
print(f"Best Baseline:   {baseline_best:.4f} ROC-AUC")

print(f"\nYour MLP:        {results_mlp['auc_mean']:.4f} ± {results_mlp['auc_std']:.4f} ROC-AUC")
print(f"Your Deep MLP:   {results_deep_mlp['auc_mean']:.4f} ± {results_deep_mlp['auc_std']:.4f} ROC-AUC")

# Improvement checks
if results_mlp['auc_mean'] > baseline_best:
    improvement = ((results_mlp['auc_mean'] - baseline_best) / baseline_best) * 100
    print(f"\n🎉 MLP improvement over best baseline: +{improvement:.2f}%")

if results_deep_mlp['auc_mean'] > baseline_best:
    improvement = ((results_deep_mlp['auc_mean'] - baseline_best) / baseline_best) * 100
    print(f"🎉 Deep MLP improvement over best baseline: +{improvement:.2f}%")

print("="*60)

# Save results
results_df.to_csv('dl_results.csv', index=False)
results_df.to_csv('/content/drive/MyDrive/emotify_colab/results/dl_results.csv', index=False)
print("\n✓ Results saved to dl_results.csv")

STARTING TRAINING

🚀 Training Model 1/2...

Training MLP (3-layer)

Fold 1/10...
  Acc: 0.1500, AUC: 0.5323, F1: 0.0731

Fold 2/10...
  Acc: 0.2500, AUC: 0.5774, F1: 0.1031

Fold 3/10...
  Acc: 0.2000, AUC: 0.5880, F1: 0.0667

Fold 4/10...
  Acc: 0.1250, AUC: 0.5514, F1: 0.0707

Fold 5/10...
  Acc: 0.1750, AUC: 0.5965, F1: 0.0724

Fold 6/10...
  Acc: 0.1000, AUC: 0.5344, F1: 0.0468

Fold 7/10...
  Acc: 0.1000, AUC: 0.5469, F1: 0.0342

Fold 8/10...
  Acc: 0.2750, AUC: 0.6205, F1: 0.1102

Fold 9/10...
  Acc: 0.1750, AUC: 0.6151, F1: 0.0799

Fold 10/10...
  Acc: 0.1282, AUC: 0.6551, F1: 0.0294

MLP (3-layer) - FINAL RESULTS
Accuracy:  0.1678 ± 0.0568
ROC-AUC:   0.5818 ± 0.0389
F1-Score:  0.0686 ± 0.0251

🚀 Training Model 2/2...

Training Deep MLP (5-layer + BatchNorm)

Fold 1/10...
  Acc: 0.0750, AUC: 0.5497, F1: 0.0504

Fold 2/10...
  Acc: 0.2000, AUC: 0.5650, F1: 0.1320

Fold 3/10...
  Acc: 0.2000, AUC: 0.5343, F1: 0.1154

Fold 4/10...
  Acc: 0.2000, AUC: 0.4482, F1: 0.0828

Fold 5/10..

In [ ]:
 from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.preprocessing import label_binarize
import numpy as np


In [ ]:
# Random Forest
print("\n Training Random Forest")

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_split=2,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

rf_aucs, rf_f1s = [], []

le = LabelEncoder()
y_enc = le.fit_transform(y)
classes = np.unique(y_enc)

for train_idx, test_idx in skf.split(X, y_enc):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y_enc[train_idx], y_enc[test_idx]

    rf.fit(X_train, y_train)
    y_proba = rf.predict_proba(X_test)
    y_pred = rf.predict(X_test)

    y_test_bin = label_binarize(y_test, classes=classes)

    rf_aucs.append(
        roc_auc_score(y_test_bin, y_proba, average='macro', multi_class='ovr')
    )
    rf_f1s.append(
        f1_score(y_test, y_pred, average='macro')
    )

print(f"RF ROC-AUC: {np.mean(rf_aucs):.4f} ± {np.std(rf_aucs):.4f}")
print(f"RF F1:      {np.mean(rf_f1s):.4f}")



 Training Random Forest
RF ROC-AUC: 0.5120 ± 0.0419
RF F1:      0.0953


In [ ]:
print("\n Training Extra Trees")

et = ExtraTreesClassifier(
    n_estimators=500,
    max_depth=None,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

et_aucs, et_f1s = [], []

for train_idx, test_idx in skf.split(X, y_enc):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y_enc[train_idx], y_enc[test_idx]

    et.fit(X_train, y_train)
    y_proba = et.predict_proba(X_test)
    y_pred = et.predict(X_test)

    y_test_bin = label_binarize(y_test, classes=classes)

    et_aucs.append(
        roc_auc_score(y_test_bin, y_proba, average='macro', multi_class='ovr')
    )
    et_f1s.append(
        f1_score(y_test, y_pred, average='macro')
    )

print(f"ET ROC-AUC: {np.mean(et_aucs):.4f} ± {np.std(et_aucs):.4f}")
print(f"ET F1:      {np.mean(et_f1s):.4f}")



 Training Extra Trees
ET ROC-AUC: 0.5434 ± 0.0520
ET F1:      0.0934


In [ ]:
#Installing CatBoost
!pip install -q catboost
print("✓ CatBoost installed")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.8 MB/s eta 0:00:00
✓ CatBoost installed


In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.metrics import roc_auc_score, f1_score
import numpy as np


In [ ]:
print("\n Training CatBoost")

# Encode labels
le = LabelEncoder()
y_enc = le.fit_transform(y)
classes = np.unique(y_enc)

# 10-fold CV
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

cb_aucs, cb_f1s = [], []

for fold, (train_idx, test_idx) in enumerate(skf.split(X, y_enc), 1):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y_enc[train_idx], y_enc[test_idx]

    model = CatBoostClassifier(
        iterations=500,
        depth=6,
        learning_rate=0.05,
        loss_function='MultiClass',
        eval_metric='MultiClass',
        verbose=False,
        random_seed=42
    )

    model.fit(X_train, y_train)

    y_proba = model.predict_proba(X_test)
    y_pred = model.predict(X_test).ravel()

    y_test_bin = label_binarize(y_test, classes=classes)

    auc = roc_auc_score(
        y_test_bin,
        y_proba,
        average='macro',
        multi_class='ovr'
    )
    f1 = f1_score(y_test, y_pred, average='macro')

    cb_aucs.append(auc)
    cb_f1s.append(f1)

    print(
        f"Fold {fold:2d} | "
        f"AUC: {auc:.4f} | "
        f"F1: {f1:.4f}"
    )

print("\n" + "-" * 60)
print("CATBOOST FINAL RESULTS")
print("-" * 60)
print(f"ROC-AUC: {np.mean(cb_aucs):.4f} ± {np.std(cb_aucs):.4f}")
print(f"F1:      {np.mean(cb_f1s):.4f}")



 Training CatBoost
Fold  1 | AUC: 0.4764 | F1: 0.0516
Fold  2 | AUC: 0.5527 | F1: 0.0916
Fold  3 | AUC: 0.5075 | F1: 0.0980
Fold  4 | AUC: 0.5025 | F1: 0.1288
Fold  5 | AUC: 0.5485 | F1: 0.1036
Fold  6 | AUC: 0.5041 | F1: 0.1062
Fold  7 | AUC: 0.5171 | F1: 0.0976
Fold  8 | AUC: 0.6587 | F1: 0.1591
Fold  9 | AUC: 0.5493 | F1: 0.2229
Fold 10 | AUC: 0.6128 | F1: 0.1313

------------------------------------------------------------
CATBOOST FINAL RESULTS
------------------------------------------------------------
ROC-AUC: 0.5430 ± 0.0529
F1:      0.1191


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, label_binarize
from sklearn.metrics import roc_auc_score, f1_score
import numpy as np

In [ ]:
#Logistic Regression

print("\n Training Logistic Regression")

# Encode labels
le = LabelEncoder()
y_enc = le.fit_transform(y)
classes = np.unique(y_enc)

# Scale features (important for LR)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

lr_aucs, lr_f1s = [], []

for fold, (train_idx, test_idx) in enumerate(skf.split(X_scaled, y_enc), 1):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y_enc[train_idx], y_enc[test_idx]

    lr = LogisticRegression(
        max_iter=2000,
        multi_class='multinomial',
        solver='lbfgs',
        class_weight='balanced',
        n_jobs=-1
    )

    lr.fit(X_train, y_train)

    y_proba = lr.predict_proba(X_test)
    y_pred = lr.predict(X_test)

    y_test_bin = label_binarize(y_test, classes=classes)

    auc = roc_auc_score(
        y_test_bin,
        y_proba,
        average='macro',
        multi_class='ovr'
    )
    f1 = f1_score(y_test, y_pred, average='macro')

    lr_aucs.append(auc)
    lr_f1s.append(f1)

    print(f"Fold {fold:2d} | AUC: {auc:.4f} | F1: {f1:.4f}")

print("\n" + "-" * 60)
print("LOGISTIC REGRESSION FINAL RESULTS")
print("-" * 60)
print(f"ROC-AUC: {np.mean(lr_aucs):.4f} ± {np.std(lr_aucs):.4f}")
print(f"F1:      {np.mean(lr_f1s):.4f}")



 Training Logistic Regression


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Fold  1 | AUC: 0.5106 | F1: 0.0786


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Fold  2 | AUC: 0.5387 | F1: 0.1631
Fold  3 | AUC: 0.4721 | F1: 0.0167
Fold  4 | AUC: 0.4085 | F1: 0.1137


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Fold  5 | AUC: 0.5236 | F1: 0.0843
Fold  6 | AUC: 0.5248 | F1: 0.1308
Fold  7 | AUC: 0.5540 | F1: 0.0984


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Fold  8 | AUC: 0.4892 | F1: 0.0582
Fold  9 | AUC: 0.5595 | F1: 0.1041
Fold 10 | AUC: 0.5327 | F1: 0.0652

------------------------------------------------------------
LOGISTIC REGRESSION FINAL RESULTS
------------------------------------------------------------
ROC-AUC: 0.5114 ± 0.0427
F1:      0.0913


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [ ]:
from sklearn.naive_bayes import GaussianNB


In [ ]:
print("\n Training Naive Bayes (Gaussian)")

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

nb_aucs, nb_f1s = [], []

for fold, (train_idx, test_idx) in enumerate(skf.split(X_scaled, y_enc), 1):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y_enc[train_idx], y_enc[test_idx]

    nb = GaussianNB()
    nb.fit(X_train, y_train)

    y_proba = nb.predict_proba(X_test)
    y_pred = nb.predict(X_test)

    y_test_bin = label_binarize(y_test, classes=classes)

    auc = roc_auc_score(
        y_test_bin,
        y_proba,
        average='macro',
        multi_class='ovr'
    )
    f1 = f1_score(y_test, y_pred, average='macro')

    nb_aucs.append(auc)
    nb_f1s.append(f1)

    print(f"Fold {fold:2d} | AUC: {auc:.4f} | F1: {f1:.4f}")

print("\n" + "-" * 60)
print("NAIVE BAYES FINAL RESULTS")
print("-" * 60)
print(f"ROC-AUC: {np.mean(nb_aucs):.4f} ± {np.std(nb_aucs):.4f}")
print(f"F1:      {np.mean(nb_f1s):.4f}")



 Training Naive Bayes (Gaussian)
Fold  1 | AUC: 0.5542 | F1: 0.1600
Fold  2 | AUC: 0.4262 | F1: 0.0615
Fold  3 | AUC: 0.5145 | F1: 0.1404
Fold  4 | AUC: 0.5198 | F1: 0.0962
Fold  5 | AUC: 0.5170 | F1: 0.1190
Fold  6 | AUC: 0.5022 | F1: 0.0404
Fold  7 | AUC: 0.4518 | F1: 0.0487
Fold  8 | AUC: 0.5692 | F1: 0.0782
Fold  9 | AUC: 0.5278 | F1: 0.0694
Fold 10 | AUC: 0.5538 | F1: 0.0957

------------------------------------------------------------
NAIVE BAYES FINAL RESULTS
------------------------------------------------------------
ROC-AUC: 0.5137 ± 0.0427
F1:      0.0910


# Deep Learning Model Training

In [ ]:
!pip install -q librosa soundfile
print("✓ Audio libraries installed")

✓ Audio libraries installed


In [ ]:
!ls /content/drive/MyDrive/emotify_colab

code  data  results


In [ ]:
!ls /content/drive/MyDrive/emotify_colab/data

audio  emotify_features.csv  emotify_labels_matched.csv


In [ ]:
!ls /content/drive/MyDrive/emotify_colab/data/audio/emotify/classical

100.mp3  19.mp3  28.mp3  37.mp3  46.mp3  55.mp3  64.mp3  73.mp3  82.mp3  91.mp3
10.mp3	 1.mp3	 29.mp3  38.mp3  47.mp3  56.mp3  65.mp3  74.mp3  83.mp3  92.mp3
11.mp3	 20.mp3  2.mp3	 39.mp3  48.mp3  57.mp3  66.mp3  75.mp3  84.mp3  93.mp3
12.mp3	 21.mp3  30.mp3  3.mp3	 49.mp3  58.mp3  67.mp3  76.mp3  85.mp3  94.mp3
13.mp3	 22.mp3  31.mp3  40.mp3  4.mp3	 59.mp3  68.mp3  77.mp3  86.mp3  95.mp3
14.mp3	 23.mp3  32.mp3  41.mp3  50.mp3  5.mp3	 69.mp3  78.mp3  87.mp3  96.mp3
15.mp3	 24.mp3  33.mp3  42.mp3  51.mp3  60.mp3  6.mp3	 79.mp3  88.mp3  97.mp3
16.mp3	 25.mp3  34.mp3  43.mp3  52.mp3  61.mp3  70.mp3  7.mp3	 89.mp3  98.mp3
17.mp3	 26.mp3  35.mp3  44.mp3  53.mp3  62.mp3  71.mp3  80.mp3  8.mp3	 99.mp3
18.mp3	 27.mp3  36.mp3  45.mp3  54.mp3  63.mp3  72.mp3  81.mp3  90.mp3  9.mp3


In [ ]:
import os

audio_base = '/content/drive/MyDrive/emotify_colab/data/audio/emotify'

print("Audio base exists:", os.path.exists(audio_base))
print("Pop samples:", os.listdir(os.path.join(audio_base, 'pop'))[:5])


Audio base exists: True
Pop samples: ['3.mp3', '6.mp3', '9.mp3', '13.mp3', '11.mp3']


In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import layers, models

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Audio parameters
SAMPLE_RATE = 22050
N_MELS = 128
N_FFT = 2048
HOP_LENGTH = 512
MAX_DURATION = 10  # seconds


In [ ]:
audio_base = '/content/drive/MyDrive/emotify_colab/data/audio/emotify'

print("Audio base exists:", os.path.exists(audio_base))
print("Genres:", os.listdir(audio_base))
print("Pop samples:", os.listdir(os.path.join(audio_base, 'pop'))[:5])


Audio base exists: True
Genres: ['rock', 'pop', 'electronic', 'classical']
Pop samples: ['3.mp3', '6.mp3', '9.mp3', '13.mp3', '11.mp3']


In [ ]:
def augment_audio(y):
    # Random volume gain
    gain = np.random.uniform(0.8, 1.2)
    y = y * gain

    # Add small Gaussian noise
    noise = np.random.randn(len(y)) * 0.005
    y = y + noise

    return y


def extract_logmel(audio_path, augment=False):
    y, sr = librosa.load(audio_path, sr=SAMPLE_RATE, duration=MAX_DURATION)

    if augment:
        y = augment_audio(y)

    max_len = SAMPLE_RATE * MAX_DURATION
    if len(y) < max_len:
        y = np.pad(y, (0, max_len - len(y)))
    else:
        y = y[:max_len]

    mel = librosa.feature.melspectrogram(
        y=y,
        sr=sr,
        n_fft=N_FFT,
        hop_length=HOP_LENGTH,
        n_mels=N_MELS
    )

    logmel = librosa.power_to_db(mel)
    logmel = (logmel - np.mean(logmel)) / (np.std(logmel) + 1e-8)

    return logmel


In [ ]:
labels_df = pd.read_csv(
    '/content/drive/MyDrive/emotify_colab/data/emotify_labels_matched.csv'
)

X_specs = []
y_labels = []

for _, row in tqdm(labels_df.iterrows(), total=len(labels_df)):
    audio_path = os.path.join(audio_base, row['filename'])

    if not os.path.exists(audio_path):
        continue

    spec = extract_logmel(audio_path, augment = True)
    X_specs.append(spec)
    y_labels.append(row['dominant_emotion'])

100%|██████████| 399/399 [04:03<00:00,  1.64it/s]


In [ ]:
 X = np.array(X_specs)[..., np.newaxis]  # (N, 128, T, 1)

le = LabelEncoder()
y = le.fit_transform(y_labels)

print("X shape:", X.shape)
print("y shape:", y.shape)
print("Classes:", len(np.unique(y)))


X shape: (399, 128, 431, 1)
y shape: (399,)
Classes: 10


In [ ]:
# CNN Model (Baseline)
def build_cnn(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(64, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(128, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),

        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
    return model

In [ ]:
# Train/Validation Split
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

model = build_cnn(X.shape[1:], len(np.unique(y)))


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Training
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=16
)

Epoch 1/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 23s 377ms/step - accuracy: 0.0616 - loss: 2.3442 - val_accuracy: 0.1375 - val_loss: 2.3068
Epoch 2/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.1380 - loss: 2.2585 - val_accuracy: 0.1250 - val_loss: 2.3052
Epoch 3/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.0990 - loss: 2.2958 - val_accuracy: 0.1250 - val_loss: 2.3058
Epoch 4/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.1992 - loss: 2.2359 - val_accuracy: 0.1250 - val_loss: 2.3065
Epoch 5/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.1907 - loss: 2.2165 - val_accuracy: 0.1250 - val_loss: 2.3029
Epoch 6/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - accuracy: 0.2108 - loss: 2.2399 - val_accuracy: 0.1250 - val_loss: 2.2977
Epoch 7/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.1959 - loss: 2.1971 - val_accuracy: 0.1250 - val_loss: 2.2970
Epoch 8/30
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.1930 - loss: 2.1914 - val_accuracy: 0.1250 -

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 429, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 126, 429, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 214, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 212, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 61, 212, 64)    │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 106, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 104, 128)   │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 104, 128)   │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 333,216 (1.27 MB)

 Trainable params: 110,922 (433.29 KB)

 Non-trainable params: 448 (1.75 KB)

 Optimizer params: 221,846 (866.59 KB)

In [ ]:
# =================================================================
# IMPROVED CNN - RESEARCH-GRADE FIXES
# =================================================================

import os
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.utils.class_weight import compute_class_weight

# =================================================================
# PARAMETERS
# =================================================================
SAMPLE_RATE = 22050
N_MELS = 128
N_FFT = 2048
HOP_LENGTH = 512
SEGMENT_DURATION = 30  # 30-second segments (not full 60s)
SEGMENTS_PER_SONG = 3  # Extract 3 random segments per song

audio_base = '/content/drive/MyDrive/emotify_colab/data/audio/emotify'
labels_path = '/content/drive/MyDrive/emotify_colab/data/emotify_labels_matched.csv'

# =================================================================
# FIX 1: SEGMENT-LEVEL EXTRACTION (BIGGEST FIX!)
# =================================================================
def extract_random_segment(audio_path, segment_duration=30):
    """Extract a random segment from the audio"""
    y, sr = librosa.load(audio_path, sr=SAMPLE_RATE, duration=60)

    segment_len = int(SAMPLE_RATE * segment_duration)

    # Random crop if long enough
    if len(y) > segment_len:
        start = np.random.randint(0, len(y) - segment_len)
        y = y[start:start + segment_len]
    else:
        y = np.pad(y, (0, segment_len - len(y)))

    # Extract mel-spectrogram
    mel = librosa.feature.melspectrogram(
        y=y, sr=sr,
        n_fft=N_FFT,
        hop_length=HOP_LENGTH,
        n_mels=N_MELS
    )

    logmel = librosa.power_to_db(mel)
    logmel = (logmel - np.mean(logmel)) / (np.std(logmel) + 1e-8)

    return logmel

# =================================================================
# LOAD DATA WITH MULTIPLE SEGMENTS PER SONG
# =================================================================
print("Loading data with segment-level extraction...")
labels_df = pd.read_csv(labels_path)

X_specs = []
y_labels = []

for _, row in tqdm(labels_df.iterrows(), total=len(labels_df), desc="Extracting segments"):
    audio_path = os.path.join(audio_base, row['filename'])

    if not os.path.exists(audio_path):
        continue

    # Extract multiple segments from each song
    for _ in range(SEGMENTS_PER_SONG):
        spec = extract_random_segment(audio_path, SEGMENT_DURATION)
        X_specs.append(spec)
        y_labels.append(row['dominant_emotion'])

X = np.array(X_specs)[..., np.newaxis]
le = LabelEncoder()
y = le.fit_transform(y_labels)

print(f"\nData loaded:")
print(f"X shape: {X.shape}")  # Should be ~1200 samples now!
print(f"y shape: {y.shape}")
print(f"Classes: {len(np.unique(y))}")
print(f"Data multiplication: {len(X) / len(labels_df):.1f}x")

# =================================================================
# CNN ARCHITECTURE (keeping GAP - it's actually good!)
# =================================================================
def build_cnn(input_shape, num_classes):
    model = models.Sequential([
        # Block 1
        layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.25),

        # Block 2
        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.25),

        # Block 3
        layers.Conv2D(128, (3,3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.3),

        # Global pooling + Dense
        layers.GlobalAveragePooling2D(),  # Keeping this!
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.4),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# =================================================================
# FIX 2 & 4: 10-FOLD CV WITH CLASS WEIGHTS & ROC-AUC
# =================================================================
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

fold_aucs = []
fold_f1s = []
fold_accs = []

print("\n" + "="*60)
print("TRAINING IMPROVED CNN (Segment-level + Class Weights)")
print("="*60)

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
    print(f"\nFold {fold}/10")

    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # FIX 2: Compute class weights
    class_weights_array = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_train),
        y=y_train
    )
    class_weights = dict(enumerate(class_weights_array))

    # Build model
    model = build_cnn(X.shape[1:], len(np.unique(y)))

    # Callbacks
    early_stop = callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=0
    )

    reduce_lr = callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=0
    )

    # Train with class weights
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=100,
        batch_size=32,
        class_weight=class_weights,  # FIX 2: Class weights!
        callbacks=[early_stop, reduce_lr],
        verbose=0
    )

    # FIX 4: Evaluate with ROC-AUC!
    y_pred = model.predict(X_val, verbose=0)
    y_pred_classes = np.argmax(y_pred, axis=1)

    acc = accuracy_score(y_val, y_pred_classes)
    f1 = f1_score(y_val, y_pred_classes, average='macro')

    # ROC-AUC (proper comparison with ML models)
    y_val_bin = label_binarize(y_val, classes=range(len(le.classes_)))
    auc = roc_auc_score(y_val_bin, y_pred, average='macro', multi_class='ovr')

    fold_accs.append(acc)
    fold_f1s.append(f1)
    fold_aucs.append(auc)

    print(f"Acc: {acc:.4f} | AUC: {auc:.4f} | F1: {f1:.4f} | Epochs: {len(history.history['loss'])}")

# =================================================================
# FINAL RESULTS
# =================================================================
print("\n" + "="*60)
print("IMPROVED CNN RESULTS")
print("="*60)
print(f"Accuracy:  {np.mean(fold_accs):.4f} ± {np.std(fold_accs):.4f}")
print(f"ROC-AUC:   {np.mean(fold_aucs):.4f} ± {np.std(fold_aucs):.4f}")
print(f"F1-Score:  {np.mean(fold_f1s):.4f} ± {np.std(fold_f1s):.4f}")

print("\n" + "="*60)
print("COMPARISON WITH BASELINES")
print("="*60)
print(f"KNN:              0.52 ROC-AUC")
print(f"SVM:              0.53 ROC-AUC")
print(f"MLP:              0.57 ROC-AUC")
print(f"Deep MLP:         0.58 ROC-AUC")
print(f"CNN (improved):   {np.mean(fold_aucs):.4f} ROC-AUC")

if np.mean(fold_aucs) > 0.58:
    improvement = ((np.mean(fold_aucs) - 0.58) / 0.58) * 100
    print(f"\n🎉 CNN beats best baseline by +{improvement:.1f}%!")
print("="*60)

Loading data with segment-level extraction...


Extracting segments: 100%|██████████| 399/399 [04:41<00:00,  1.42it/s]



Data loaded:
X shape: (1197, 128, 1292, 1)
y shape: (1197,)
Classes: 10
Data multiplication: 3.0x

TRAINING IMPROVED CNN (Segment-level + Class Weights)

Fold 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Acc: 0.1667 | AUC: 0.5527 | F1: 0.0288 | Epochs: 17

Fold 2/10


...

Acc: 0.1092 | AUC: 0.5068 | F1: 0.0350 | Epochs: 16

Fold 9/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Acc: 0.1345 | AUC: 0.5239 | F1: 0.0375 | Epochs: 17

Fold 10/10


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Acc: 0.1176 | AUC: 0.5104 | F1: 0.0212 | Epochs: 16

IMPROVED CNN RESULTS
Accuracy:  0.1236 ± 0.0200
ROC-AUC:   0.5249 ± 0.0199
F1-Score:  0.0332 ± 0.0094

COMPARISON WITH BASELINES
KNN:              0.52 ROC-AUC
SVM:              0.53 ROC-AUC
MLP:              0.57 ROC-AUC
Deep MLP:         0.58 ROC-AUC
CNN (improved):   0.5249 ROC-AUC


LSTM

In [ ]:
# =================================================================
# LSTM FOR EMOTIFY+ MER
# =================================================================

import os
import numpy as np
import pandas as pd
import librosa
from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.utils.class_weight import compute_class_weight

# =================================================================
# PARAMETERS
# =================================================================
SAMPLE_RATE = 22050
N_MELS = 128
N_FFT = 2048
HOP_LENGTH = 512
MAX_DURATION = 30  # 30 seconds

audio_base = '/content/drive/MyDrive/emotify_colab/data/audio/emotify'
labels_path = '/content/drive/MyDrive/emotify_colab/data/emotify_labels_matched.csv'

# =================================================================
# FEATURE EXTRACTION FOR LSTM (TIME SERIES)
# =================================================================
def extract_mel_sequence(audio_path, duration=30):
    """Extract mel-spectrogram as a time sequence"""
    y, sr = librosa.load(audio_path, sr=SAMPLE_RATE, duration=60)

    segment_len = int(SAMPLE_RATE * duration)

    # Random crop
    if len(y) > segment_len:
        start = np.random.randint(0, len(y) - segment_len)
        y = y[start:start + segment_len]
    else:
        y = np.pad(y, (0, segment_len - len(y)))

    # Extract mel-spectrogram
    mel = librosa.feature.melspectrogram(
        y=y, sr=sr,
        n_fft=N_FFT,
        hop_length=HOP_LENGTH,
        n_mels=N_MELS
    )

    logmel = librosa.power_to_db(mel)

    # Normalize per feature (across time)
    logmel = (logmel - np.mean(logmel, axis=1, keepdims=True)) / (np.std(logmel, axis=1, keepdims=True) + 1e-8)

    # Transpose: (time_steps, features)
    logmel = logmel.T  # Shape: (time_steps, 128)

    return logmel

# =================================================================
# LOAD DATA WITH SEGMENTS
# =================================================================
print("Extracting mel sequences for LSTM...")
labels_df = pd.read_csv(labels_path)

SEGMENTS_PER_SONG = 3  # 3 segments per song

X_seqs = []
y_labels = []

for _, row in tqdm(labels_df.iterrows(), total=len(labels_df), desc="Extracting sequences"):
    audio_path = os.path.join(audio_base, row['filename'])

    if not os.path.exists(audio_path):
        continue

    # Extract multiple segments
    for _ in range(SEGMENTS_PER_SONG):
        seq = extract_mel_sequence(audio_path, MAX_DURATION)
        X_seqs.append(seq)
        y_labels.append(row['dominant_emotion'])

X = np.array(X_seqs)  # Shape: (N, time_steps, 128)
le = LabelEncoder()
y = le.fit_transform(y_labels)

print(f"\nData loaded:")
print(f"X shape: {X.shape}")  # Should be (1197, time_steps, 128)
print(f"y shape: {y.shape}")
print(f"Classes: {len(np.unique(y))}")

# =================================================================
# LSTM MODEL ARCHITECTURES
# =================================================================

def build_lstm(input_shape, num_classes):
    """Simple LSTM"""
    model = models.Sequential([
        layers.Input(shape=input_shape),

        layers.LSTM(128, return_sequences=True),
        layers.Dropout(0.3),

        layers.LSTM(128, return_sequences=False),
        layers.Dropout(0.3),

        layers.Dense(64, activation='relu'),
        layers.Dropout(0.4),

        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

def build_bilstm(input_shape, num_classes):
    """Bidirectional LSTM"""
    model = models.Sequential([
        layers.Input(shape=input_shape),

        layers.Bidirectional(layers.LSTM(128, return_sequences=True)),
        layers.Dropout(0.3),

        layers.Bidirectional(layers.LSTM(64, return_sequences=False)),
        layers.Dropout(0.3),

        layers.Dense(128, activation='relu'),
        layers.Dropout(0.4),

        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

def build_gru(input_shape, num_classes):
    """GRU (faster alternative to LSTM)"""
    model = models.Sequential([
        layers.Input(shape=input_shape),

        layers.GRU(128, return_sequences=True),
        layers.Dropout(0.3),

        layers.GRU(128, return_sequences=False),
        layers.Dropout(0.3),

        layers.Dense(64, activation='relu'),
        layers.Dropout(0.4),

        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# =================================================================
# TRAIN ALL RNN MODELS WITH 10-FOLD CV
# =================================================================

models_to_train = {
    'LSTM': build_lstm,
    'BiLSTM': build_bilstm,
    'GRU': build_gru
}

all_results = {}

for model_name, model_builder in models_to_train.items():

    print("\n" + "="*60)
    print(f"TRAINING {model_name}")
    print("="*60)

    skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    fold_aucs = []
    fold_f1s = []
    fold_accs = []

    for fold, (train_idx, val_idx) in enumerate(skf.split(X, y), 1):
        print(f"\nFold {fold}/10", end=" ")

        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        # Class weights
        class_weights_array = compute_class_weight(
            class_weight='balanced',
            classes=np.unique(y_train),
            y=y_train
        )
        class_weights = dict(enumerate(class_weights_array))

        # Build model
        model = model_builder(X.shape[1:], len(np.unique(y)))

        # Callbacks
        early_stop = callbacks.EarlyStopping(
            monitor='val_loss',
            patience=20,
            restore_best_weights=True,
            verbose=0
        )

        reduce_lr = callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=7,
            min_lr=1e-6,
            verbose=0
        )

        # Train
        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=100,
            batch_size=32,
            class_weight=class_weights,
            callbacks=[early_stop, reduce_lr],
            verbose=0
        )

        # Evaluate
        y_pred = model.predict(X_val, verbose=0)
        y_pred_classes = np.argmax(y_pred, axis=1)

        acc = accuracy_score(y_val, y_pred_classes)
        f1 = f1_score(y_val, y_pred_classes, average='macro')

        # ROC-AUC
        y_val_bin = label_binarize(y_val, classes=range(len(le.classes_)))
        auc = roc_auc_score(y_val_bin, y_pred, average='macro', multi_class='ovr')

        fold_accs.append(acc)
        fold_f1s.append(f1)
        fold_aucs.append(auc)

        print(f"| Acc: {acc:.4f} | AUC: {auc:.4f} | F1: {f1:.4f}")

    # Store results
    all_results[model_name] = {
        'accuracy': (np.mean(fold_accs), np.std(fold_accs)),
        'roc_auc': (np.mean(fold_aucs), np.std(fold_aucs)),
        'f1': (np.mean(fold_f1s), np.std(fold_f1s))
    }

    print(f"\n{model_name} Results:")
    print(f"  Accuracy:  {np.mean(fold_accs):.4f} ± {np.std(fold_accs):.4f}")
    print(f"  ROC-AUC:   {np.mean(fold_aucs):.4f} ± {np.std(fold_aucs):.4f}")
    print(f"  F1-Score:  {np.mean(fold_f1s):.4f} ± {np.std(fold_f1s):.4f}")

# =================================================================
# FINAL SUMMARY
# =================================================================
print("\n" + "="*60)
print("COMPLETE RESULTS SUMMARY")
print("="*60)

print("\nML Baselines:")
print("  KNN:              0.5200 ROC-AUC")
print("  SVM:              0.5300 ROC-AUC")
print("  Extra Trees:      0.5400 ROC-AUC")
print("  MLP:              0.5700 ROC-AUC")
print("  Deep MLP:         0.5800 ROC-AUC")

print("\nDeep Learning:")
print(f"  CNN:              0.5249 ROC-AUC")

for model_name, results in all_results.items():
    auc_mean, auc_std = results['roc_auc']
    print(f"  {model_name:17s} {auc_mean:.4f} ± {auc_std:.4f} ROC-AUC")

# Find best model
best_model = max(all_results.items(), key=lambda x: x[1]['roc_auc'][0])
best_auc = best_model[1]['roc_auc'][0]

print("\n" + "="*60)
print(f"BEST MODEL: {best_model[0]} with {best_auc:.4f} ROC-AUC")

if best_auc > 0.58:
    improvement = ((best_auc - 0.58) / 0.58) * 100
    print(f"🎉 Improvement over Deep MLP: +{improvement:.1f}%")
else:
    print("Note: RNNs perform similarly to feature-based models")
    print("This confirms that on small datasets, hand-crafted features remain competitive")

print("="*60)

# Save results
results_df = pd.DataFrame([
    {'Model': k,
     'ROC_AUC': f"{v['roc_auc'][0]:.4f} ± {v['roc_auc'][1]:.4f}",
     'F1': f"{v['f1'][0]:.4f} ± {v['f1'][1]:.4f}"}
    for k, v in all_results.items()
])
results_df.to_csv('rnn_results.csv', index=False)
print("\n✓ Results saved to: rnn_results.csv")

Extracting mel sequences for LSTM...


Extracting sequences: 100%|██████████| 399/399 [13:20<00:00,  2.01s/it]



Data loaded:
X shape: (1197, 1292, 128)
y shape: (1197,)
Classes: 10

TRAINING LSTM

Fold 1/10 | Acc: 0.1000 | AUC: 0.5701 | F1: 0.0838

Fold 2/10 | Acc: 0.1083 | AUC: 0.5674 | F1: 0.0828

Fold 3/10 

| Acc: 0.1917 | AUC: 0.5698 | F1: 0.1729

Fold 4/10 | Acc: 0.1000 | AUC: 0.4965 | F1: 0.0795

Fold 5/10 | Acc: 0.1500 | AUC: 0.5636 | F1: 0.1613

Fold 6/10 | Acc: 0.1417 | AUC: 0.5332 | F1: 0.1140

Fold 7/10 | Acc: 0.1167 | AUC: 0.5538 | F1: 0.1111

Fold 8/10 | Acc: 0.1176 | AUC: 0.4886 | F1: 0.1147

Fold 9/10 | Acc: 0.1176 | AUC: 0.5797 | F1: 0.1015

Fold 10/10 | Acc: 0.1008 | AUC: 0.5654 | F1: 0.0863

LSTM Results:
  Accuracy:  0.1244 ± 0.0276
  ROC-AUC:   0.5488 ± 0.0305
  F1-Score:  0.1108 ± 0.0310

TRAINING BiLSTM

Fold 1/10 | Acc: 0.1333 | AUC: 0.5630 | F1: 0.1110

Fold 2/10 | Acc: 0.1250 | AUC: 0.5378 | F1: 0.1196

Fold 3/10 | Acc: 0.0750 | AUC: 0.5515 | F1: 0.0709

Fold 4/10 | Acc: 0.1500 | AUC: 0.5777 | F1: 0.1264

Fold 5/10 | Acc: 0.1417 | AUC: 0.5994 | F1: 0.1458

Fold 6/10 | Acc: 0.1167 | AUC: 0.5295 | F1: 0.0848

Fold 7/10 | Acc: 0.1000 | AUC: 0.5144 | F1: 0.0978

Fold 8/10 | Acc: 0.1008 | AUC: 0.5237 | F1: 0.0884

Fold 9/10 | Acc: 0.0672 | AUC: 0.5291 | F1: 0.0514

Fold 1